In [ ]:
import flopy as fp
import geopandas as gp 
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import Point
from pathlib import Path
import numpy as np
import pyemu
import platform, shutil
from flopy.export.shapefile_utils import recarray2shp
from flopy.utils.geometry import Polygon


### Set a couple paths to information we will need

In [ ]:
#### Base directory for runs
parent_run_path = Path("../../LPR_pycap_opt/pycap_runs")

#### depletion potential calculations directory
base_run_path = parent_run_path / "pycap_base"

#### modflow base path
modflow_run_path = Path('../steady_state_mf6')

#### modflow depletion potential calculations directory
modflow_dp_path = Path('../steady_state_mf6_DP')


### read in the depletion potetial as calculated by `pycap`

In [ ]:
dp = gp.read_file(base_run_path / 'depletion_potential.json').to_crs(3071)

### now read in the MODFLOW-6 version of the steady-state Little Plover River Model

In [ ]:
sim = fp.mf6.MFSimulation.load(sim_ws = '../steady_state_mf6')
m = sim.get_model()

### We will need a `modelgrid` object to locate wells and gages on the MODFLOW grid

In [ ]:
modelgrid = fp.discretization.StructuredGrid(xoff=1795400*.3048,  
                                             yoff=1424400*.3048,
                                             botm=m.dis.botm.array,
                                             delr = m.dis.delr.array*.3048,
                                             delc = m.dis.delc.array*.3048,
                                                
                                             crs=3071)


### We can look at the main boundaries

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
pmv = fp.plot.PlotMapView(m, ax=ax)
# lc = pmv.plot_grid(lw=0.5)
pmv.plot_bc("WEL", plotAll=True, color='red')
pmv.plot_bc("SFR", plotAll=True, color='blue')
pmv.plot_bc("CHD", plotAll=True, color='black')
ax.set_xlabel(f'{modelgrid.units.capitalize()} easting, {modelgrid.crs.name}')
ax.set_ylabel(f'{modelgrid.units.capitalize()} northing, {modelgrid.crs.name}')

#### In the figure above, the wells (showing up as kinda faint dots) are red, streams are blue, and constant head boundaries are black. Let's look at these on a map.

### how does the active domain look?

In [ ]:
idomain = [[int(i) for i in j.split()] for j in open('../steady_state_mf6/lpr_ss_gwf.dis_idomain_layer1.txt','r').readlines()]
idomain = np.array([k for j in idomain for k in j]).reshape(m.dis.nrow.array,m.dis.ncol.array)
plt.imshow(idomain)
plt.colorbar()
plt.title('IDOMAIN: 1 means active, 0 means inactive');

### Now we need to write out a few shapefiles and then we can explore the model 

In [ ]:
modelgrid.write_shapefile('grid.shp') # this is the whole grid. it's a bit cubersome to
                                    # viz though, so we will dissolve it to show only the model outline
grid_shp = gp.read_file('grid.shp').set_crs(3071)
mf6outline = grid_shp.dissolve()

In [ ]:
# next let's export the major packages
chds = pd.read_csv('../steady_state_mf6/lpr_ss_gwf.chd_stress_period_data_1.txt', sep=r'\s+', header=None, names = ['k','i','j','head'])
for cc in ['k','i','j']:
    chds[cc]-=1
chds

chd_shp = [Polygon(modelgrid.get_cell_vertices(row, col)) for row, col in zip(chds.i, chds.j)]
recarray2shp(chds.to_records(), geoms=chd_shp, shpname='../steady_state_mf6/chds.shp', crs=modelgrid.crs)
chd_map = gp.read_file('../steady_state_mf6/chds.shp')
chd_map = chd_map.set_crs(modelgrid.crs)

### Now check out the blue constant heads on the map. What do these represent? Note - the black box is the outline of the entire model, compare with the IDOMAIN plot above. 

In [ ]:
mmap = mf6outline.explore(
 style_kwds={
        "color": "black",   # outline color
        "weight": 2,        # outline thickness (pixels)
        "fillOpacity": 0    # no fill
    },highlight=False
)
chd_map.explore(m=mmap,
               style_kwds={
        "color": "blue",   # outline color
        "fillOpacity": .1    # no fill
    })


## We can now look at the Depletion Potential and set up some comparisons

### note the depletion potential color scale bar - the size also represents depletion potential magnitude

In [ ]:
dp.explore(column="Depletion_Potential",
                  vmin=0,vmax=1,m=mmap,
                  style_kwds={"style_function":
                                  lambda x: 
                                  {"radius":x["properties"]["Depletion_Potential"]*15}})


# Let's run the base model and have a look at budget components

In [ ]:
# RUN! Should take bewteen 2 and 6 minutes
pyemu.os_utils.run('mf6', cwd = str(modflow_run_path))

In [ ]:
# Open a text editor and check out the two `.lst` files in the following location
print(modflow_run_path)

# We can also use `flopy` to read in the budget information (there are a couple ways to do this - here showing the easiest way)

In [ ]:
flux, vol = m.output.list().get_dataframes()

In [ ]:
flux

# We can check this out in a stacked bar chart as well

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
in_cols = ['WEL_IN', 'RCHA_IN', 'CHD_IN', 'SFR_IN', 'DRN_IN']
out_cols = [c.replace('_IN', '_OUT') for c in in_cols]
flux[in_cols].plot.bar(stacked=True, ax=ax)
(-flux[out_cols]).plot.bar(stacked=True, ax=ax)
ax.legend(loc='lower left', bbox_to_anchor=(1, 0))
ax.axhline(0, lw=0.5, c='k')
ax.set_ylabel('Simulated Flux, in $m^3/d$')